# AMHE Centroid Influence

__Richard Staszkiewicz__

__Igor Matynia__

The notebook is supposed to be run on Google Collaboratory. It explores the influence the different centroid counting has on the DES algorithm performnnce. The algorithm is faced with an optimization problem of the chosen benchmark functions accessed with curtasy of [Axel Thevenot](https://github.com/AxelThevenot/Python_Benchmark_Test_Optimization_Function_Single_Objective).

In [ ]:
!https://github.com/AxelThevenot/Python_Benchmark_Test_Optimization_Function_Single_Objective.git